In [ ]:
# imports
from encryption import encryption

encryp = encryption()
file = "image/test.png"
message = "En un pequeño pueblo rodeado de montañas, la vida transcurría con una calma casi mágica. Cada mañana, los rayos del sol se filtraban entre los árboles, creando un espectáculo de luces y sombras que despertaba a los habitantes. Los niños corrían por las calles empedradas, riendo y jugando, mientras las abuelas se sentaban en los bancos de la plaza, compartiendo historias de tiempos pasados. La comunidad se unía cada domingo en el mercado local, donde los agricultores ofrecían sus productos frescos y los artesanos exhibían sus creaciones. La música de un guitarrista resonaba en el aire, llenando el ambiente de alegría y nostalgia. A medida que el día avanzaba, el aroma del pan recién horneado y las comidas caseras se mezclaban, invitando a todos a disfrutar de una comida en compañía. Así, en este rincón del mundo, la simplicidad de la vida cotidiana se convertía en un verdadero tesoro, recordando a cada uno la importancia de la conexión con los demás y con la naturaleza."

In [ ]:
# unit test
secretKey = encryp.encrypt(message, file)
# encryp.seeImage()

message2 = encryp.decrypt(file, secretKey)
print(message2)


In [ ]:
# count = 0

# for i in range(100):
#     try:
#         secretKey = encryp.encrypt(message, file)
#     except ValueError as e:
#         raise e
    
#     try:
#         message2 = encryp.decrypt(file, secretKey)
#     except ValueError as e:
#         raise e
    
#     if message != message2:
#         raise ValueError("Decrypted message does not match original message")
    
#     count += 1
#     if count % 10 == 0:
#         print(count)